We can connect to our device using the android tool adb

You should have an adb server deamon running already with root

In [11]:
# Lets note what partitions we have on the device
!tools/adb shell "cat /proc/partitions" > device_info/partitions.proc.info

In [1]:
cat device_info/partitions.proc.info

In [13]:
# Lets note the nameing conventions in the device
!tools/adb shell "ls -al /dev/block/by-name" > device_info/by-name.block.dev.info

In [2]:
cat device_info/by-name.block.dev.info

In [5]:
# Now lets pull each partition into our tablet directory only pull one at a time

!tools/adb shell "dd if=/dev/block/nanda 2>/dev/null" | dd of=tablet/nanda.boot-resource.img

65526+90 records in
65552+1 records out
33562881 bytes (34 MB) copied, 7.84205 s, 4.3 MB/s


In [6]:
!tools/adb shell "dd if=/dev/block/nandb 2>/dev/null" | dd of=tablet/nandb.env.img

32768+0 records in
32768+0 records out
16777216 bytes (17 MB) copied, 3.99497 s, 4.2 MB/s


In [7]:
!tools/adb shell "dd if=/dev/block/nandc 2>/dev/null" | dd of=tablet/nandc.boot.img

32450+1088 records in
32941+1 records out
16866141 bytes (17 MB) copied, 5.90316 s, 2.9 MB/s


In [8]:
!tools/adb shell "dd if=/dev/block/nandd 2>/dev/null" | dd of=tablet/nandd.system.img

1562816+35293 records in
1577750+1 records out
807808272 bytes (808 MB) copied, 244.414 s, 3.3 MB/s


In [9]:
!tools/adb shell "dd if=/dev/block/nande 2>/dev/null" | dd of=tablet/nande.data.img

2086471+59203 records in
2111711+1 records out
1081196242 bytes (1.1 GB) copied, 400.796 s, 2.7 MB/s


In [10]:
!tools/adb shell "dd if=/dev/block/nandf 2>/dev/null" | dd of=tablet/nandf.misc.img

32768+0 records in
32768+0 records out
16777216 bytes (17 MB) copied, 4.18592 s, 4.0 MB/s


In [11]:
!tools/adb shell "dd if=/dev/block/nandg 2>/dev/null" | dd of=tablet/nandg.recovery.img

65017+1626 records in
65735+1 records out
33656486 bytes (34 MB) copied, 10.2956 s, 3.3 MB/s


In [12]:
!tools/adb shell "dd if=/dev/block/nandh 2>/dev/null" | dd of=tablet/nandh.cache.img

262151+5 records in
262151+1 records out
134221551 bytes (134 MB) copied, 31.61 s, 4.2 MB/s


In [13]:
!tools/adb shell "dd if=/dev/block/nandi 2>/dev/null" | dd of=tablet/nandi.databk.img

262144+3 records in
262144+1 records out
134217762 bytes (134 MB) copied, 31.7368 s, 4.2 MB/s


In [14]:
!tools/adb shell "dd if=/dev/block/nandj 2>/dev/null" | dd of=tablet/nandj.UDISK.img

9338571+99970 records in
9381891+1 records out
4803528369 bytes (4.8 GB) copied, 1518.96 s, 3.2 MB/s


In [16]:
# We should now have a backup or each partition
!ls -lGg tablet/*.img

-rwxrwxrwx 1   33562881 Mar 14 14:22 tablet/nanda.boot-resource.img
-rwxrwxrwx 1   16777216 Mar 14 14:23 tablet/nandb.env.img
-rwxrwxrwx 1   16866141 Mar 14 14:24 tablet/nandc.boot.img
-rwxrwxrwx 1  807808272 Mar 14 14:30 tablet/nandd.system.img
-rwxrwxrwx 1 1081196242 Mar 14 14:38 tablet/nande.data.img
-rwxrwxrwx 1   16777216 Mar 14 14:41 tablet/nandf.misc.img
-rwxrwxrwx 1   33656486 Mar 14 14:41 tablet/nandg.recovery.img
-rwxrwxrwx 1  134221551 Mar 14 14:42 tablet/nandh.cache.img
-rwxrwxrwx 1  134217762 Mar 14 14:47 tablet/nandi.databk.img
-rwxrwxrwx 1 4803528369 Mar 14 15:13 tablet/nandj.UDISK.img


In [17]:
# Lets archive what we've got and can refer to the originals without having to dump again
!tar zcvf org/Allwinner_Polaris_A70H.tar.gz tablet/*.img

tablet/nanda.boot-resource.img
tablet/nandb.env.img
tablet/nandc.boot.img
tablet/nandd.system.img
tablet/nande.data.img
tablet/nandf.misc.img
tablet/nandg.recovery.img
tablet/nandh.cache.img
tablet/nandi.databk.img
tablet/nandj.UDISK.img
